In [ ]:
!pip uninstall transformers
!pip install git+https://github.com/huggingface/transformers
!pip install accelerate

Found existing installation: transformers 4.51.3
Uninstalling transformers-4.51.3:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.11/dist-packages/transformers-4.51.3.dist-info/*
    /usr/local/lib/python3.11/dist-packages/transformers/*
Proceed (Y/n)? y
  Successfully uninstalled transformers-4.51.3
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-i14ar2he
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-i14ar2he
  Resolved https://github.com/huggingface/transformers to commit 7a3e208892c06a5e278144eaf38c8599a42f53e7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.52.0.dev0-py3-none-any.whl size=11588256 sha256=89914f42bd12ab786a83b66ce6568e88c02395d5c109c24e32b0f7bf02087c78
  Stored in directory: /tmp/pip-ephem-whee

In [ ]:
!pip install qwen-omni-utils[decord] -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.2/35.2 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 71.6 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-1.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('data.csv')

In [ ]:
essays = []
# good
for row, column in df[df['Overall'] == 5.0].sample(4, random_state=42).iterrows():
  essays.append(column['full_text'].replace('\n\n', ' ').strip())
for row, column in df[df['Overall'] == 3.0].sample(3, random_state=42).iterrows():
  essays.append(column['full_text'].replace('\n\n', ' ').strip())
for row, column in df[df['Overall'] == 1.0].sample(3, random_state=42).iterrows():
  essays.append(column['full_text'].replace('\n\n', ' ').strip())

In [ ]:
def get_feedback(essay, temperature):
  prompt = f"""You are given the following essay.

Provide feedback in a structured and concise format, addressing ONLY the following aspects:
- Overall
- Cohesion
- Syntax
- Vocabulary
- Phraseology
- Grammar and Conventions

For each aspect:
- Explicitly note possible improvements if applicable
- DO provide a general impression or summary inside overall aspect part.

{essay}
"""

  messages = [
    {"role": "system", "content": "You are a helpful English language teaching assistant."},
    {"role": "user", "content": prompt},
]

  text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    temperature=temperature
)

  model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

  generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048
)

  generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

  response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

  return response

In [ ]:
feedbacks = []
for essay in essays:
  feedbacks.append(get_feedback(essay, 0.5))

In [ ]:
with open('feedbacks.txt', 'w', encoding='utf-8') as f:
  for i in range(len(feedbacks)):
    f.write(essays[i][:50] + '\n' + feedbacks[i] + '\n\n\n\n\n')